In [48]:
import time
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import MaxNLocator
OUTDIR = "figures"
os.makedirs(OUTDIR, exist_ok=True)

def save_fig(fig, filename, dpi=300):
    path = os.path.join(OUTDIR, filename)
    fig.savefig(path, dpi=dpi, bbox_inches="tight")
    print(f"[saved] {path}")
# -----------------------------
# Parameters
# -----------------------------
C1 = 1e-4          # Armijo/Wolfe c1
C2 = 2e-1          # Wolfe parameter
GAMMA_PEN = 1.0    # penalty parameter gamma
NEWTON_TOL = 1e-8  # Newton tolerance
NEWTON_MAXIT = 50  # safe cap
LS_MAXIT_WOLFE = 20
LS_KMAX_ARMIJO = 20
SIGMA = 0.05      # 
np.set_printoptions(precision=4, suppress=True)
# -----------------------------
def build_structured_mesh(n_sub):
    h = 1.0 / n_sub
    xs = np.linspace(0.0, 1.0, n_sub+1)
    ys = np.linspace(0.0, 1.0, n_sub+1)
    X, Y = np.meshgrid(xs, ys, indexing="xy")
    coords = np.column_stack([X.ravel(), Y.ravel()])

    def nid(i, j):  # i along x, j along y
        return j*(n_sub+1) + i

    tris = []
    for j in range(n_sub):
        for i in range(n_sub):
            n00 = nid(i, j)
            n10 = nid(i+1, j)
            n01 = nid(i, j+1)
            n11 = nid(i+1, j+1)
            tris.append([n00, n10, n11])
            tris.append([n00, n11, n01])
    tris = np.array(tris, dtype=int)
    tol = 1e-15
    interior = (
        (coords[:,0] > tol) & (coords[:,0] < 1.0-tol) &
        (coords[:,1] > tol) & (coords[:,1] < 1.0-tol)
    )
    map_int = -np.ones(coords.shape[0], dtype=int)
    int_nodes = np.where(interior)[0]
    map_int[int_nodes] = np.arange(int_nodes.size)
    return coords, tris, interior, map_int
def assemble_MA(coords, tris, map_int):
    ndof = map_int.max() + 1
    rowsM, colsM, dataM = [], [], []
    rowsA, colsA, dataA = [], [], []
    for t in tris:
        g = t
        loc = map_int[g]   
        # coordinates
        x = coords[g, 0]
        y = coords[g, 1]
        mat = np.array([[1, x[0], y[0]],[1, x[1], y[1]], [1, x[2], y[2]]], dtype=float)
        det = np.linalg.det(mat)
        area = abs(det) / 2.0
        if area <= 0:
            continue
        b = np.array([y[1]-y[2], y[2]-y[0], y[0]-y[1]], dtype=float)
        c = np.array([x[2]-x[1], x[0]-x[2], x[1]-x[0]], dtype=float)
        grad = np.column_stack([b, c]) / (2.0*area)  # shape (3,2)
        Ke = area * (grad @ grad.T)  # (3,3)
        Me = (area / 12.0) * np.array([[2,1,1],[1,2,1],[1,1,2]], dtype=float)
        for i in range(3):
            Ii = loc[i]
            if Ii < 0:
                continue
            for j in range(3):
                Jj = loc[j]
                if Jj < 0:
                    continue
                rowsM.append(Ii); colsM.append(Jj); dataM.append(Me[i,j])
                rowsA.append(Ii); colsA.append(Jj); dataA.append(Ke[i,j])
    M = sp.csr_matrix((dataM, (rowsM, colsM)), shape=(ndof, ndof))
    A = sp.csr_matrix((dataA, (rowsA, colsA)), shape=(ndof, ndof))
    M = 0.5*(M + M.T)
    A = 0.5*(A + A.T)
    return M, A
# -----------------------------
# Initial condition y0 
# -----------------------------
def gaussian_y0(coords_int, sigma=SIGMA):
    x = coords_int[:,0]
    y = coords_int[:,1]
    r2 = (x-0.5)**2 + (y-0.5)**2
    base = np.exp(-r2/(2*sigma**2))
    damp = x*(1-x)*y*(1-y)
    y0 = damp * base
    m = y0.max()
    if m > 0:
        y0 = y0 / m
    return y0
# -----------------------------
# State and adjoint equations
# -----------------------------
def solve_stateANTIGUO(M, A, dt, u_time, y0, newton_tol=NEWTON_TOL, newton_maxit=NEWTON_MAXIT):
    m, ndof = u_time.shape
    y = np.zeros((m+1, ndof))
    y[0] = y0.copy()
    I = sp.eye(ndof, format="csr")
    for k in range(m):
        uk = u_time[k]
        y_prev = y[k]
        # Newton method
        w = y_prev.copy()  
        Mu = M @ sp.diags(uk, format="csr")  
        Kbase = (M + dt*A + dt*Mu - dt*M)    
        for it in range(newton_maxit):
            w2 = w*w
            F = Kbase.dot(w) + dt*(M.dot(w2)) - M.dot(y_prev)
            J = Kbase + (2.0*dt)*(M @ sp.diags(w, format="csr"))
            try:
                dw = spla.spsolve(J, -F)
            except Exception:
                dw = spla.lgmres(J, -F, atol=0, tol=1e-12)[0]
            w_new = w + dw
            denom = np.linalg.norm(w_new) + 1e-30
            if np.linalg.norm(dw) / denom <= newton_tol:
                w = w_new
                break
            w = w_new
        y[k+1] = w
    return y
def solve_state(M, A, dt, u_time, y0, newton_tol=NEWTON_TOL, newton_maxit=NEWTON_MAXIT, clip_u_for_pde=True):
    m, ndof = u_time.shape
    y = np.zeros((m+1, ndof))
    y[0] = y0.copy()
    ok = True
    for k in range(m):
        uk = u_time[k]
        if clip_u_for_pde:
            uk = np.clip(uk, 0.0, 1.0)
        y_prev = y[k]
        w = y_prev.copy()
        Mu = M @ sp.diags(uk, format="csr")
        Kbase = (M + dt*A + dt*Mu - dt*M)
        converged = False
        for it in range(newton_maxit):
            w2 = w*w
            F = Kbase.dot(w) + dt*(M.dot(w2)) - M.dot(y_prev)
            J = Kbase + (2.0*dt)*(M @ sp.diags(w, format="csr"))
            try:
                dw = spla.spsolve(J, -F)
            except Exception:
                dw = spla.lgmres(J, -F, atol=0, tol=1e-12)[0]
            if not np.all(np.isfinite(dw)):
                break
            w_new = w + dw
            denom = np.linalg.norm(w_new) + 1e-30
            if np.linalg.norm(dw) / denom <= newton_tol:
                w = w_new
                converged = True
                break
            w = w_new
        if not converged or (not np.all(np.isfinite(w))):
            ok = False
            y[k+1] = w
            break
        y[k+1] = w
    return y,ok

def solve_adjoint(M, A, dt, u_time, y_time, y_target):
    m, ndof = u_time.shape
    p = np.zeros((m+1, ndof))
    p[m] = M.dot(y_time[m] - y_target)
    for k in range(m-1, -1, -1):
        uk = u_time[k]
        ykp1 = y_time[k+1]
        Mu = M @ sp.diags(uk, format="csr")
        G = (M + dt*A + dt*Mu - dt*M) + (2.0*dt)*(M @ sp.diags(ykp1, format="csr"))
        rhs = M.dot(p[k+1])
        try:
            p[k] = spla.spsolve(G, rhs)
        except Exception:
            p[k] = spla.lgmres(G, rhs, atol=0, tol=1e-12)[0]
    return p
# -----------------------------
# Objective, penalties, gradient
# -----------------------------
def unpack_u(u_vec, m, ndof):
    return u_vec.reshape(m, ndof)

def pack_u(u_time):
    return u_time.reshape(-1)
def penalty_terms(M, dt, gamma, u_time):
    v1 = np.maximum(-u_time, 0.0)
    v2 = np.maximum(u_time - 1.0, 0.0)
    phi1 = 0.0
    phi2 = 0.0
    for k in range(u_time.shape[0]):
        phi1 += 0.5*gamma*dt * (v1[k].T @ (M.dot(v1[k])))
        phi2 += 0.5*gamma*dt * (v2[k].T @ (M.dot(v2[k])))
    return phi1, phi2
def objectiveANTIGUO(M, A, dt, gamma, u_time, y0, y_target):
    y_time = solve_state(M, A, dt, u_time, y0)
    ym = y_time[-1]
    term = 0.5 * (ym - y_target).T @ (M.dot(ym - y_target))

    reg = 0.0
    for k in range(u_time.shape[0]):
        reg += 0.5*dt * (u_time[k].T @ (M.dot(u_time[k])))

    phi1, phi2 = penalty_terms(M, dt, gamma, u_time)
    return term + reg + phi1 + phi2, y_time
def gradientANTIGUO(M, A, dt, gamma, u_time, y0, y_target):
    y_time = solve_state(M, A, dt, u_time, y0)
    p_time = solve_adjoint(M, A, dt, u_time, y_time, y_target)

    m, ndof = u_time.shape
    g = np.zeros((m, ndof))
    for k in range(m):
        pkp1 = p_time[k+1]
        ykp1 = y_time[k+1]
        u = u_time[k]
        term1 = - M.dot(pkp1 * ykp1)               
        term2 = dt * M.dot(u)
        term3 = - dt*gamma * M.dot(np.maximum(-u, 0.0))
        term4 = + dt*gamma * M.dot(np.maximum(u - 1.0, 0.0))
        g[k] = term1 + term2 + term3 + term4
    return g, y_time, p_time
def objective(M, A, dt, gamma, u_time, y0, y_target):
    y_time, ok = solve_state(M, A, dt, u_time, y0, clip_u_for_pde=True)
    if not ok:
        return np.inf, y_time
    ym = y_time[-1]
    term = 0.5 * (ym - y_target).T @ (M.dot(ym - y_target))
    reg = 0.0
    for k in range(u_time.shape[0]):
        reg += 0.5*dt * (u_time[k].T @ (M.dot(u_time[k])))
    phi1, phi2 = penalty_terms(M, dt, gamma, u_time)
    f = term + reg + phi1 + phi2
    if not np.isfinite(f):
        return np.inf, y_time
    return f, y_time
def gradient(M, A, dt, gamma, u_time, y0, y_target):
    y_time, ok = solve_state(M, A, dt, u_time, y0, clip_u_for_pde=True)
    if not ok:
        return None, y_time, None
    p_time = solve_adjoint(M, A, dt, np.clip(u_time,0,1), y_time, y_target) 
    m, ndof = u_time.shape
    g = np.zeros((m, ndof))
    for k in range(m):
        pkp1 = p_time[k+1]
        ykp1 = y_time[k+1]
        u = u_time[k]
        term1 = - M.dot(pkp1 * ykp1)
        term2 = dt * M.dot(u)
        term3 = - dt*gamma * M.dot(np.maximum(-u, 0.0))
        term4 = + dt*gamma * M.dot(np.maximum(u - 1.0, 0.0))
        gk = term1 + term2 + term3 + term4
        if not np.all(np.isfinite(gk)):
            return None, y_time, p_time
        g[k] = gk
    return g, y_time, p_time
# -----------------------------
# Line search
# -----------------------------
def armijo_line_search(M, A, dt, gamma, u_time, y0, y_target, d_time, g_time, f0, kmax=LS_KMAX_ARMIJO, c1=C1):
    gdotd = float(np.sum(g_time * d_time))
    if gdotd >= 0:
        return 0.0, f0, None
    for k in range(kmax):
        alpha = 2.0**(-k)
        u_try = u_time + alpha*d_time
        f_try, y_try = objective(M, A, dt, gamma, u_try, y0, y_target)
        if f_try <= f0 + c1*alpha*gdotd:
            return alpha, f_try, y_try
    return 0.0, f0, None
def wolfe_backtrackingANTIGUO(M, A, dt, gamma, u_time, y0, y_target, d_time, g_time, f0, maxit=LS_MAXIT_WOLFE, c1=C1, c2=C2):
    alpha = 1.0
    gdotd = float(np.sum(g_time * d_time))
    if gdotd >= 0:
        return 0.0, f0, None, None  # not descent
    for _ in range(maxit):
        u_try = u_time + alpha*d_time
        f_try, _ = objective(M, A, dt, gamma, u_try, y0, y_target)
        if f_try > f0 + c1*alpha*gdotd:
            alpha *= 0.5
            continue
        g_try, _, _ = gradient(M, A, dt, gamma, u_try, y0, y_target)
        gtry_dotd = float(np.sum(g_try * d_time))
        if gtry_dotd >= c2*gdotd:
            return alpha, f_try, u_try, g_try
        alpha *= 0.5
    return 0.0, f0, None, None
def wolfe_backtracking(M, A, dt, gamma, u_time, y0, y_target, d_time, g_time, f0,
                      maxit=LS_MAXIT_WOLFE, c1=C1, c2=C2, fallback_armijo=True):
    alpha = 1.0
    gdotd = float(np.sum(g_time * d_time))
    if not np.isfinite(gdotd) or gdotd >= 0:
        return 0.0, f0, None, None 
    for _ in range(maxit):
        u_try = u_time + alpha*d_time
        f_try, _ = objective(M, A, dt, gamma, u_try, y0, y_target)
        if not np.isfinite(f_try):
            alpha *= 0.5
            continue
        # Armijo condition
        if f_try > f0 + c1*alpha*gdotd:
            alpha *= 0.5
            continue
        g_try, _, _ = gradient(M, A, dt, gamma, u_try, y0, y_target)
        if g_try is None:
            alpha *= 0.5
            continue
        gtry_dotd = float(np.sum(g_try * d_time))
        if not np.isfinite(gtry_dotd):
            alpha *= 0.5
            continue
        if gtry_dotd >= c2*gdotd:
            return alpha, f_try, u_try, g_try
        alpha *= 0.5
    if fallback_armijo:
        alpha2, f2, y2 = armijo_line_search(M, A, dt, gamma, u_time, y0, y_target, d_time, g_time, f0,
                                            kmax=LS_KMAX_ARMIJO, c1=c1)
        if alpha2 > 0:
            u2 = u_time + alpha2*d_time
            g2, _, _ = gradient(M, A, dt, gamma, u2, y0, y_target)
            if g2 is not None:
                return alpha2, f2, u2, g2
    return 0.0, f0, None, None
# -----------------------------
# Optimization methods
# -----------------------------
def steepest_descent(M, A, dt, gamma, u0_time, y0, y_target, tol=1e-4, maxit=5000, kmax=LS_KMAX_ARMIJO):
    u = u0_time.copy()
    history = {"err": [], "f": [], "alpha": [], "time": 0.0, "iters": 0}
    t0 = time.perf_counter()
    f0, _ = objective(M, A, dt, gamma, u, y0, y_target)
    for it in range(maxit):
        g, _, _ = gradient(M, A, dt, gamma, u, y0, y_target)
        d = -g
        err = np.linalg.norm(d.ravel(), 2)
        history["err"].append(err)
        history["f"].append(f0)
        if err < tol:
            history["iters"] = it
            break
        alpha, f_new, y_new = armijo_line_search(M, A, dt, gamma, u, y0, y_target, d, g, f0, kmax=kmax)
        history["alpha"].append(alpha)
        if alpha == 0.0:
            history["iters"] = it
            break
        u = u + alpha*d
        f0 = f_new
    history["time"] = time.perf_counter() - t0
    return u, history
def sr1_method(M, A, dt, gamma, u0_time, y0, y_target, tol=1e-4, maxit=100, ls_maxit=LS_MAXIT_WOLFE):
    u = u0_time.copy()
    m, ndof = u.shape
    N = m*ndof
    Hinv = np.eye(N)  
    history = {"err": [], "f": [], "alpha": [], "time": 0.0, "iters": 0}
    t0 = time.perf_counter()
    f0, _ = objective(M, A, dt, gamma, u, y0, y_target)
    g_time, _, _ = gradient(M, A, dt, gamma, u, y0, y_target)
    g = g_time.reshape(N)
    for it in range(maxit):
        d = -(Hinv @ g)
        d_time = d.reshape(m, ndof)
        err = np.linalg.norm(g, 2)
        history["err"].append(err)
        history["f"].append(f0)
        if err < tol:
            history["iters"] = it
            break
        alpha, f_new, u_new, g_new_time = wolfe_backtracking(
            M, A, dt, gamma, u, y0, y_target, d_time, g_time, f0, maxit=ls_maxit
        )
        history["alpha"].append(alpha)
        if alpha == 0.0:
            history["iters"] = it
            break
        s = (u_new - u).reshape(N)
        g_new = g_new_time.reshape(N)
        yk = g_new - g
        Hy = Hinv @ yk
        v = s - Hy
        denom = float(v @ yk)
        if abs(denom) > 1e-12:
            Hinv = Hinv + np.outer(v, v) / denom
        u = u_new
        g = g_new
        g_time = g_new_time
        f0 = f_new
    history["time"] = time.perf_counter() - t0
    return u, history
def bfgs_method(M, A, dt, gamma, u0_time, y0, y_target, tol=1e-4, maxit=100, ls_maxit=LS_MAXIT_WOLFE):
    u = u0_time.copy()
    m, ndof = u.shape
    N = m*ndof
    Hinv = np.eye(N)
    history = {"err": [], "f": [], "alpha": [], "time": 0.0, "iters": 0}
    t0 = time.perf_counter()
    f0, _ = objective(M, A, dt, gamma, u, y0, y_target)
    g_time, _, _ = gradient(M, A, dt, gamma, u, y0, y_target)
    g = g_time.reshape(N)
    for it in range(maxit):
        d = -(Hinv @ g)
        d_time = d.reshape(m, ndof)
        err = np.linalg.norm(g, 2)
        history["err"].append(err)
        history["f"].append(f0)
        if err < tol:
            history["iters"] = it
            break
        alpha, f_new, u_new, g_new_time = wolfe_backtracking( M, A, dt, gamma, u, y0, y_target, d_time, g_time, f0, maxit=ls_maxit)
        history["alpha"].append(alpha)
        if alpha == 0.0:
            history["iters"] = it
            break
        s = (u_new - u).reshape(N)
        g_new = g_new_time.reshape(N)
        yk = g_new - g
        ys = float(yk @ s)
        if ys > 1e-12:
            rho = 1.0 / ys
            I = np.eye(N)
            V = (I - rho*np.outer(s, yk))
            Hinv = V @ Hinv @ V.T + rho*np.outer(s, s)
        u = u_new
        g = g_new
        g_time = g_new_time
        f0 = f_new
    history["time"] = time.perf_counter() - t0
    return u, history
# -----------------------------
# Plot
# -----------------------------
def plot_surface_from_interior(n_sub, interior_values, title):
    Nnodes_axis = n_sub + 1
    grid = np.zeros((Nnodes_axis, Nnodes_axis))
    idx = 0
    for j in range(1, n_sub):
        for i in range(1, n_sub):
            grid[j, i] = interior_values[idx]
            idx += 1
    xs = np.linspace(0, 1, Nnodes_axis)
    ys = np.linspace(0, 1, Nnodes_axis)
    X, Y = np.meshgrid(xs, ys, indexing="xy")
    fig = plt.figure(figsize=(7,5))
    ax = fig.add_subplot(111, projection="3d")
    ax.plot_surface(X, Y, grid, rstride=1, cstride=1, linewidth=0, antialiased=True)
    ax.set_title(title)
    ax.set_xlabel("x-axis")
    ax.set_ylabel("y-axis")
    return fig, ax
def plot_convergence(hist, title):
    fig = plt.figure(figsize=(6,4))
    ax = plt.gca()
    ax.plot(np.arange(1, len(hist["err"]) + 1), hist["err"])
    ax.set_xlabel("Iterations")
    ax.set_ylabel(r"$e_k$")
    ax.set_title(title)
    ax.grid(True)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    return fig
# -----------------------------
# Running
# -----------------------------
def run_case(n_SPACE, m, method_set=("SD_Armijo", "SR1_Armijo", "BFGS_Armijo", "SR1_Wolfe", "BFGS_Wolfe"),
             tol=1e-4, bfgs_tol_override=None):
    n_sub = n_SPACE + 1
    dt = 1.0 / m
    coords, tris, interior, map_int = build_structured_mesh(n_sub)
    coords_int = coords[interior]
    M, A = assemble_MA(coords, tris, map_int)
    ndof = M.shape[0]
    assert ndof == n_SPACE*n_SPACE, f"ndof mismatch: got {ndof}, expected {n_SPACE*n_SPACE}"
    y0 = gaussian_y0(coords_int, sigma=SIGMA)
    y_target = 0.5*np.ones(ndof)
    u0_time = 0.2*np.ones((m, ndof))
    results = {}
    # SD + Armijo
    if "SD_Armijo" in method_set:
        u_sd, h_sd = steepest_descent(M, A, dt, GAMMA_PEN, u0_time, y0, y_target, tol=tol, maxit=6000, kmax=LS_KMAX_ARMIJO)
        results["SD_Armijo"] = (u_sd, h_sd)
    if "SR1_Armijo" in method_set:
        u_sr1, h_sr1 = sr1_method(M, A, dt, GAMMA_PEN, u0_time, y0, y_target, tol=tol, maxit=50, ls_maxit=LS_MAXIT_WOLFE)
        results["SR1_Armijo_like"] = (u_sr1, h_sr1)
    if "BFGS_Armijo" in method_set:
        u_bfgs, h_bfgs = bfgs_method(M, A, dt, GAMMA_PEN, u0_time, y0, y_target, tol=tol, maxit=50, ls_maxit=LS_MAXIT_WOLFE)
        results["BFGS_Armijo_like"] = (u_bfgs, h_bfgs)
    if "SR1_Wolfe" in method_set:
        u_sr1w, h_sr1w = sr1_method(M, A, dt, GAMMA_PEN, u0_time, y0, y_target, tol=tol, maxit=50, ls_maxit=LS_MAXIT_WOLFE)
        results["SR1_Wolfe"] = (u_sr1w, h_sr1w)
    if "BFGS_Wolfe" in method_set:
        btol = tol if bfgs_tol_override is None else bfgs_tol_override
        u_bfgsw, h_bfgsw = bfgs_method(M, A, dt, GAMMA_PEN, u0_time, y0, y_target, tol=btol, maxit=100, ls_maxit=LS_MAXIT_WOLFE)
        results["BFGS_Wolfe"] = (u_bfgsw, h_bfgsw)
    key = "BFGS_Wolfe" if "BFGS_Wolfe" in results else list(results.keys())[0]
    u_star = results[key][0]
    y_time, ok = solve_state(M, A, dt, u_star, y0, clip_u_for_pde=True)
    if not ok:
        raise RuntimeError("State solve failed.")
    p_time = solve_adjoint(M, A, dt, np.clip(u_star,0,1), y_time, y_target)
    meta = {
        "n_SPACE": n_SPACE, "n_sub": n_sub, "m": m, "dt": dt,
        "ndof": ndof, "N_total": ndof*m,
        "y0": y0, "y_target": y_target,
        "y_time_star": y_time, "p_time_star": p_time,
        "M": M, "A": A, "coords_int": coords_int
    }
    return results, meta
def armijo_line_search_safe(M, A, dt, gamma, u_time, y0, y_target,
                            d_time, g_time, f0,
                            kmax=LS_KMAX_ARMIJO, c1=C1):
    gdotd = float(np.sum(g_time * d_time))
    if (not np.isfinite(gdotd)) or (gdotd >= 0):
        return 0.0, f0, None, None

    for k in range(kmax):
        alpha = 2.0**(-k)
        u_try = u_time + alpha*d_time

        f_try, _ = objective(M, A, dt, gamma, u_try, y0, y_target)
        if not np.isfinite(f_try):
            continue

        if f_try <= f0 + c1*alpha*gdotd:
            g_try, _, _ = gradient(M, A, dt, gamma, u_try, y0, y_target)
            if g_try is not None:
                return alpha, f_try, u_try, g_try

    return 0.0, f0, None, None
def sr1_method_armijo(M, A, dt, gamma, u0_time, y0, y_target,
                      tol=1e-4, maxit=200, kmax=LS_KMAX_ARMIJO):
    u = u0_time.copy()
    m, ndof = u.shape
    N = m * ndof
    Hinv = np.eye(N)
    history = {"err": [], "f": [], "alpha": [], "time": 0.0, "iters": 0}
    t0 = time.perf_counter()
    f0, _ = objective(M, A, dt, gamma, u, y0, y_target)
    g_time, _, _ = gradient(M, A, dt, gamma, u, y0, y_target)
    if g_time is None:
        raise RuntimeError("SR1-Armijo: gradient failed at initial iterate.")
    g = g_time.reshape(N)
    for it in range(maxit):
        d = -(Hinv @ g)
        err = np.linalg.norm(g, 2)
        history["err"].append(err)
        history["f"].append(f0)

        if err < tol:
            history["iters"] = it
            break
        d_time = d.reshape(m, ndof)
        alpha, f_new, u_new, g_new_time = armijo_line_search_safe( M, A, dt, gamma, u, y0, y_target, d_time, g_time, f0, kmax=kmax )
        history["alpha"].append(alpha)
        if alpha == 0.0:
            history["iters"] = it
            break
        s = (u_new - u).reshape(N)
        g_new = g_new_time.reshape(N)
        yk = g_new - g
        Hy = Hinv @ yk
        v = s - Hy
        denom = float(v @ yk)
        if abs(denom) > 1e-12:
            Hinv = Hinv + np.outer(v, v) / denom
        u = u_new
        g = g_new
        print(it, np.linalg.norm(g,2), np.linalg.norm((Hinv@g),2))
        g_time = g_new_time
        f0 = f_new
    history["time"] = time.perf_counter() - t0
    return u, history
def add_grad_norm_to_history(hist, M, A, dt, gamma, u_time, y0, y_target):
    g_time, _, _ = gradient(M, A, dt, gamma, u_time, y0, y_target)
    if g_time is None:
        return np.nan
    return float(np.linalg.norm(g_time.ravel(), 2))
def bfgs_method_armijo(M, A, dt, gamma, u0_time, y0, y_target,
                       tol=1e-4, maxit=200, kmax=LS_KMAX_ARMIJO):
    u = u0_time.copy()
    m, ndof = u.shape
    N = m * ndof
    Hinv = np.eye(N)
    history = {"err": [], "f": [], "alpha": [], "time": 0.0, "iters": 0}
    t0 = time.perf_counter()
    f0, _ = objective(M, A, dt, gamma, u, y0, y_target)
    g_time, _, _ = gradient(M, A, dt, gamma, u, y0, y_target)
    if g_time is None:
        raise RuntimeError("BFGS-Armijo: gradient failed at initial iterate.")
    g = g_time.reshape(N)
    for it in range(maxit):
        d = -(Hinv @ g)
        err = np.linalg.norm(g, 2)
        history["err"].append(err)
        history["f"].append(f0)
        if err < tol:
            history["iters"] = it
            break
        d_time = d.reshape(m, ndof)
        alpha, f_new, u_new, g_new_time = armijo_line_search_safe(M, A, dt, gamma, u, y0, y_target, d_time, g_time, f0, kmax=kmax)
        history["alpha"].append(alpha)
        if alpha == 0.0:
            history["iters"] = it
            break
        s = (u_new - u).reshape(N)
        g_new = g_new_time.reshape(N)
        yk = g_new - g
        ys = float(yk @ s)
        if ys > 1e-12:
            rho = 1.0 / ys
            I = np.eye(N)
            V = (I - rho*np.outer(s, yk))
            Hinv = V @ Hinv @ V.T + rho*np.outer(s, s)
        u = u_new
        g = g_new
        print(it, np.linalg.norm(g,2), np.linalg.norm((Hinv@g),2))
        g_time = g_new_time
        f0 = f_new
    history["time"] = time.perf_counter() - t0
    return u, history
def run_case_LINE_SEARCH_armijo(n_SPACE=10, m=10, tol=1e-4):
    n_sub = n_SPACE + 1
    dt = 1.0 / m
    coords, tris, interior, map_int = build_structured_mesh(n_sub)
    coords_int = coords[interior]
    M, A = assemble_MA(coords, tris, map_int)
    ndof = M.shape[0]
    assert ndof == n_SPACE*n_SPACE, f"ndof mismatch: got {ndof}, expected {n_SPACE*n_SPACE}"
    y0 = gaussian_y0(coords_int, sigma=SIGMA)
    y_target = 0.5*np.ones(ndof)
    u0_time = 0.2*np.ones((m, ndof))
    results = {}
    u_sd, h_sd = steepest_descent(M, A, dt, GAMMA_PEN, u0_time, y0, y_target,tol=tol, maxit=8000, kmax=LS_KMAX_ARMIJO)
    results["Steepest Descent (Armijo)"] = (u_sd, h_sd)
    u_sr1, h_sr1 = sr1_method_armijo(M, A, dt, GAMMA_PEN, u0_time, y0, y_target,tol=tol, maxit=200, kmax=LS_KMAX_ARMIJO)
    results["SR1 (Armijo)"] = (u_sr1, h_sr1)
    u_bfgs, h_bfgs = bfgs_method_armijo(M, A, dt, GAMMA_PEN, u0_time, y0, y_target,tol=tol, maxit=200, kmax=LS_KMAX_ARMIJO)
    results["BFGS (Armijo)"] = (u_bfgs, h_bfgs)
    meta = {"M": M, "A": A, "dt": dt, "n_sub": n_sub, "y0": y0, "y_target": y_target}
    return results, meta
def summarize_table1(results, label="Paper-style table"):
    print(f"\n{label}")
    print(f"{'Method':<28} {'Iterations':>10} {'Execution time (s)':>18}")
    print("-"*60)
    for k, (_, h) in results.items():
        its = h.get("iters", len(h["err"]))
        tsec = h.get("time", np.nan)
        print(f"{k:<28} {its:>10d} {tsec:>18.3f}")
# -----------------------------
# -----------------------------
# -----------------------------
# -----------------------------
# -----------------------------
# -----------------------------
# -----------------------------
# Executable
# -----------------------------
# -----------------------------
# -----------------------------
# -----------------------------
# -----------------------------
# -----------------------------
# -----------------------------
# -----------------------------
def summarize_table(results, label):
    print(f"\n{label}")
    print(f"{'Method':<30} {'Iterations':>10} {'Time (s)':>12}")
    print("-"*56)
    for k, (_, h) in results.items():
        its = h.get("iters", len(h["err"]))
        tsec = h.get("time", np.nan)
        print(f"{k:<30} {its:>10d} {tsec:>12.3f}")
res_A, meta_A = run_case_LINE_SEARCH_armijo(n_SPACE=10, m=10, tol=1e-3)
summarize_table1(res_A, "Table (n=10,m=10, Armijo)")
res_10, meta_10 = run_case(n_SPACE=10, m=10, tol=1e-3,method_set=("SD_Armijo","SR1_Wolfe","BFGS_Wolfe"))
summarize_table(res_10, "Case n=10, m=10 ")
u_sd = res_10["SD_Armijo"][0]
y_sd, ok = solve_state(meta_10["M"], meta_10["A"], meta_10["dt"], u_sd, meta_10["y0"], clip_u_for_pde=True)
if not ok:
    raise RuntimeError("State solve failed for SD solution.")
p_sd = solve_adjoint(meta_10["M"], meta_10["A"], meta_10["dt"], np.clip(u_sd,0,1), y_sd, meta_10["y_target"])
fig, ax = plot_surface_from_interior(meta_10["n_sub"], u_sd[-1],"Optimal Control")
save_fig(fig, "surf_control_t1_n10_m10_SD.png")
plt.show()  
plt.close(fig)
fig, ax = plot_surface_from_interior(meta_10["n_sub"], y_sd[-1],"Optimal State")
save_fig(fig, "surf_state_t1_n10_m10_SD.png")
plt.show()   
plt.close(fig)
fig, ax = plot_surface_from_interior(meta_10["n_sub"], p_sd[0], "Adjoint State")
save_fig(fig, "surf_adjoint_t0_n10_m10_SD.png")
plt.show()   # opcional
plt.close(fig)
fig = plot_convergence(res_10["SD_Armijo"][1],"Convergence")
save_fig(fig, "conv_SD_Armijo_n10_m10.png")
plt.show()  # opcional
plt.close(fig)
fig = plot_convergence(res_10["SR1_Wolfe"][1],"Convergence")
save_fig(fig, "conv_SR1_Wolfe_n10_m10.png")
plt.show()  # opcional
plt.close(fig)
fig = plot_convergence(res_10["BFGS_Wolfe"][1],"Convergence")
save_fig(fig, "conv_BFGS_Wolfe_n10_m10.png")
plt.show()  # opcional
plt.close(fig)
res_20_tight, meta_20_tight = run_case(n_SPACE=20, m=15, tol=1e-3,method_set=("BFGS_Wolfe",), bfgs_tol_override=1e-4)
summarize_table(res_20_tight, "Case n=20, m=15  - BFGS tighter tol=1e-3")
fig = plot_convergence(res_20_tight["BFGS_Wolfe"][1], "Convergence")
save_fig(fig, "conv_BFGS_Wolfe_n20_m15_tol1e-3.png")
plt.show()  
plt.close(fig)

0 0.0049059933568228395 6.250969640927782
1 0.00045602391260581737 0.22840778841818277
0 0.0049059933568228395 6.250969640849976
1 0.00045602391358108666 0.22840778442994492

Table (n=10,m=10, Armijo)
Method                       Iterations Execution time (s)
------------------------------------------------------------
Steepest Descent (Armijo)          2047            105.981
SR1 (Armijo)                          2              0.255
BFGS (Armijo)                         2              0.562


KeyboardInterrupt: 